In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

from ecmwfapi import ECMWFDataServer
import os
import sys
import dask
import numpy as np
from esio import download
import datetime
from esio import EsioData as ed
import pandas as pd
from datetime import timedelta

import calendar
import datetime

import os
from ecmwfapi import ECMWFDataServer

'''
Download models with sea ice forecasts within the 2s2 forecast data archive.

https://software.ecmwf.int/wiki/display/S2S/Models

'''

'\nDownload models with sea ice forecasts within the 2s2 forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/S2S/Models\n\n'

In [2]:
# meteofr always uses same version which is 2014-12-01
# from 1993 to one year ago

# the MVDs are on 1, 8, 15, 22 
dataclass='s2s'
run_type='reforecast'
cmod = 'cma'
versions = datetime.datetime(2014, 5, 1) # 

cmod = 'ncep' # has same format as cma but different version
versions = datetime.datetime(2011, 3, 1) # 

vdate = versions.strftime('%Y-%m-%d')
print('The version date for model ', cmod, ' is ',vdate)

The version date for model  ncep  is  2011-03-01


In [3]:
E = ed.EsioData.load()
main_dir = E.model_dir

In [4]:
# a starting point
mod_dicts = {}

mod_dicts['cma'] = {
 "class": "s2",
 "dataset": "s2s",
 "date": "2014-05-01",                                                               
 "hdate": "2005-07-11",                                                          
 "expver": "prod",
 "levtype": "sfc",
 "model": "glob",
 "origin": "babj",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
 "stream": "enfh",
 "number": "1/2/3",
 "time": "00:00:00",
 "type": "pf",
 "target": "output",
 }

mod_dicts['ncep'] = {
 "class": "s2",
 "dataset": "s2s",
 "date": "2014-05-01",                                                               
 "hdate": "2005-07-11",                                                          
 "expver": "prod",
 "levtype": "sfc",
 "model": "glob",
 "origin": "kwbc",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
 "stream": "enfh",
 "number": "1/2/3",
 "time": "00:00:00",
 "type": "pf",
 "target": "output"
 }
#"format": "netcdf"

In [5]:
mod_dicts

{'cma': {'class': 's2',
  'dataset': 's2s',
  'date': '2014-05-01',
  'expver': 'prod',
  'hdate': '2005-07-11',
  'levtype': 'sfc',
  'model': 'glob',
  'number': '1/2/3',
  'origin': 'babj',
  'param': '31',
  'step': '0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440',
  'stream': 'enfh',
  'target': 'output',
  'time': '00:00:00',
  'type': 'pf'},
 'ncep': {'class': 's2',
  'dataset': 's2s',
  'date': '2014-05-01',
  'expver': 'prod',
  'format': 'netcdf',
  'hdate': '2005-07-11',
  'levtype': 'sfc',
  'model': 'glob',
  'number': '1

In [6]:
@dask.delayed

def download_month(config_dict):
    # Start server
    cserver = ECMWFDataServer()
    cserver.retrieve(config_dict)
    return 1

In [7]:
# just for testing
cy=2000
month=1

DS = datetime.datetime(cy, month, 1) 
DE = datetime.datetime(cy, month+1, 1)
alldates = pd.date_range(start=DS, end=DE,freq='D')
alldates = pd.DatetimeIndex(alldates)
hdates = [x.strftime('%Y-%m-%d/') for x in alldates]
hdates="".join(hdates)
hdates=hdates[0:(len(hdates)-1)] # remove last slash
print(hdates)
print(hdates[0:4]+'_'+hdates[5:7])

2000-01-01/2000-01-02/2000-01-03/2000-01-04/2000-01-05/2000-01-06/2000-01-07/2000-01-08/2000-01-09/2000-01-10/2000-01-11/2000-01-12/2000-01-13/2000-01-14/2000-01-15/2000-01-16/2000-01-17/2000-01-18/2000-01-19/2000-01-20/2000-01-21/2000-01-22/2000-01-23/2000-01-24/2000-01-25/2000-01-26/2000-01-27/2000-01-28/2000-01-29/2000-01-30/2000-01-31/2000-02-01
2000_01


In [8]:
# super fast! cannot do this for ecmwf and ukmo give their method of
# changing version number every date

###print('OUTPUT IS NETCDF') # did not solve problem

#for cy in np.arange(1999,2011,1):
for cy in np.arange(2010,2011,1):
    X = 1
    for month in np.arange(1,13,1):
        DS = datetime.datetime(cy, month, 1) 
        if month<12:
            DE = datetime.datetime(cy, month+1, 1)
        else:
            DE = datetime.datetime(cy+1, 1, 1)
        alldates = pd.date_range(start=DS, end=DE,freq='D')
        alldates = pd.DatetimeIndex(alldates)
        hdates = [x.strftime('%Y-%m-%d/') for x in alldates]
        hdates="".join(hdates)
        hdates=hdates[0:(len(hdates)-1)] # remove last slash

        cdict = mod_dicts[cmod]
        cdict['date'] = vdate
        cdict['hdate'] = hdates
        target = os.path.join(main_dir, cmod, run_type, 'native',
                                  cmod+'_'+hdates[0:4]+'_'+hdates[5:7]+'.grib')
        cdict['target'] = target
        cdict['expect'] = 'any'
        print('output file is ',target)
        X = X + download_month(cdict)

    X.compute()

OUTPUT IS NETCDF
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_01.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_02.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_03.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_04.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_05.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_06.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_07.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_08.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_09.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_10.nc
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_11.nc
outp

Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-26 11:14:42 Request submitted
2019-07-26 11:14:42 Request id: 5d3b4312cad519611c8d0f04
2019-07-26 11:14:42 Request is submitted
2019-07-26 11:14:42 Request submitted
2019-07-26 11:14:42 Request id: 5d3b4312bbd91c6b111415dd
2019-07-26 11:14:42 Request is submitted
2019-07-26 11:14:42 Request submitted
2019-07-26 11:14:42 Request id: 5d3b4312c8ca19566ed3fc85
2019-07-26 11:14:42 Request is submitted
2019-07-26 11:14:43 Request subm

Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-26 11:14:44 Request is queued
2019-07-26 11:14:44 Request is queued
Error contacting the WebAPI, retrying in 60 seconds ...
Calling 'nice mars /tmp/1f/70/tmp-_marsTxxeRM.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190726.181347 - Welcome to MARS
mars - INFO   - 20190726.181347 - MARS Client build stamp: 20190607111047
mars - INFO   - 20190726.181347 - MARS Client version: 6.25.3
mars - INFO   - 20190726.181347 - MIR version: 1.2.5
mars - INFO   - 20190726.181347 - Using ecCodes version 2.12.6
mars - INFO   - 20190726.181347 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190726.181347 - Maximum retrieval size is 30.00 G
retrieve,origin=k

grib_to_netcdf: Version 2.13.0
grib_to_netcdf: Processing input file '/data/data03/scratch/b1/57/_mars-atls04-a562cefde8a29a7288fa0b8b7f9413f7-hEWwXe.grib'.
grib_to_netcdf: Found 3999 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/data/data03/scratch/76/40/_grib2netcdf-atls05-98f536083ae965b31b0d04811be6f4c6-ISAcAL.nc'
grib_to_netcdf: NetCDF library version: 4.4.1.1 of Mar 14 2018 13:48:26 $
grib_to_netcdf: Creating large (64 bit) file format.
grib_to_netcdf: Defining variable 'siconc'.
grib_to_netcdf: Done.
Process '['nice', 'grib_to_netcdf', '/data/data03/scratch/b1/57/_mars-atls04-a562cefde8a29a7288fa0b8b7f9413f7-hEWwXe.grib', '-o', '/data/data03/scratch/76/40/_grib2netcdf-atls05-98f536083ae965b31b0d04811be6f4c6-ISAcAL.nc', '-utime']' finished
2019-07-26 11:15:52 Request is complete
2019-07-26 11:15:52 Transfering 12.1331 Mbytes into /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010

Calling 'nice mars /tmp/c5/a4/tmp-_mars8zYxIa.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190726.181602 - Welcome to MARS
mars - INFO   - 20190726.181602 - MARS Client build stamp: 20190607111047
mars - INFO   - 20190726.181602 - MARS Client version: 6.25.3
mars - INFO   - 20190726.181602 - MIR version: 1.2.5
mars - INFO   - 20190726.181602 - Using ecCodes version 2.12.6
mars - INFO   - 20190726.181602 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190726.181602 - Maximum retrieval size is 30.00 G
retrieve,origin=kwbc,hdate=2010-02-01/2010-02-02/2010-02-03/2010-02-04/2010-02-05/2010-02-06/2010-02-07/2010-02-08/2010-02-09/2010-02-10/2010-02-11/2010-02-12/2010-02-13/2010-02-14/2010-02-15/2010-02-16/20

grib_to_netcdf: Version 2.13.0
grib_to_netcdf: Processing input file '/data/data03/scratch/93/06/_mars-atls14-98f536083ae965b31b0d04811be6f4c6-KekYxO.grib'.
grib_to_netcdf: Found 3999 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/data/data01/scratch/10/a5/_grib2netcdf-atls04-95e2cf679cd58ee9b4db4dd119a05a8d-9fnyja.nc'
grib_to_netcdf: NetCDF library version: 4.4.1.1 of Mar 14 2018 13:48:26 $
grib_to_netcdf: Creating large (64 bit) file format.
grib_to_netcdf: Defining variable 'siconc'.
grib_to_netcdf: Done.
Process '['nice', 'grib_to_netcdf', '/data/data03/scratch/93/06/_mars-atls14-98f536083ae965b31b0d04811be6f4c6-KekYxO.grib', '-o', '/data/data01/scratch/10/a5/_grib2netcdf-atls04-95e2cf679cd58ee9b4db4dd119a05a8d-9fnyja.nc', '-utime']' finished
2019-07-26 11:17:04 Request is complete
2019-07-26 11:17:04 Transfering 12.1331 Mbytes into /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010

2019-07-26 11:17:09 Transfer rate 2.48932 Mbytes/s
2019-07-26 11:17:10 Transfer rate 2.51233 Mbytes/s
mars - INFO   - 20190726.181738 - 3999 fields retrieved from 'marsth'
mars - WARN   - 20190726.181738 - Visiting database marsth : expected 0, got 3999
mars - INFO   - 20190726.181738 - Request time:  wall: 44 sec  cpu: 15 sec
mars - INFO   - 20190726.181738 -   Read from network: 108.38 Mbyte(s) in 28 sec  [3.84 Mbyte/sec]
mars - INFO   - 20190726.181738 -   Visiting marsth: wall: 44 sec
mars - INFO   - 20190726.181738 -   Post-processing: wall: 10 sec cpu: 10 sec
mars - INFO   - 20190726.181738 -   Writing to target file: 108.38 Mbyte(s) in < 1 sec [475.25 Mbyte/sec]
mars - INFO   - 20190726.181738 - Memory used: 35.66 Mbyte(s)
mars - INFO   - 20190726.181738 - No errors reported
Process '['nice', 'mars', '/tmp/0d/da/tmp-_marsryQCMv.req']' finished
Calling 'nice grib_to_netcdf /data/data04/scratch/4a/45/_mars-atls04-a82bacafb5c306db76464bc7e824bb75-XiiwRl.grib -o /data/data05/scratch